# 2-stage user-query building approach

Steps:
- STAGE 1: Extract conditions in natural language (spans from the input) corresponding to relevant metadata fields
    - Output (simplified one):
    ```python
    [
        ("contains English and French", "languages"),
        ("from Huggigface platform", "platform"),
    ]
    ```
    <span style="color:red">
        Suggestion: We may wish to experiment with the StructuredQuery logic, and instead of outputing complicated filters we would output a relationships between conditions in their natural language format. Doing this we may be able to model whatever complicated structures in between conditions...
    </span>
    

- STAGE 2: Process each natural language condition separately (we can dynamically create new output schemas)
    - Output schema (for one condition):
    ```python   
    class Condition(BaseModel):
        values: list[DYNAMIC_TYPE]
        comp: Literal[">", "<", ...]
        log_op: Literal["AND", "OR"]
    ```

Problems:
- It's not easy to model relationships (logical opetaros) between multiple Conditions associated with different types
    - I suppose for now we will not support such use cases: E.g., "Retrieve models that are either in Slovak or that have at least 1 million datapoints."
    - We implicitly apply AND operator in between the Conditions. OR operator can only be applied in between values pertaining to a specific metadata field